In [1]:
import tkinter as tk
from tkinter import messagebox
import json
import os

class QuizApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Quiz Maker")
        self.questions_file = "questions.json"
        self.questions = self.load_questions()
        self.current_question = 0
        self.score = 0

        self.setup_ui()

    def setup_ui(self):
        self.frame = tk.Frame(self.root)
        self.frame.pack(padx=20, pady=20)

        self.title_label = tk.Label(self.frame, text="Quiz Maker", font=("Arial", 20))
        self.title_label.pack(pady=10)

        self.add_question_button = tk.Button(self.frame, text="Add Question", command=self.add_question_ui)
        self.add_question_button.pack(pady=10)

        self.start_quiz_button = tk.Button(self.frame, text="Start Quiz", command=self.start_quiz, state=tk.NORMAL if self.questions else tk.DISABLED)
        self.start_quiz_button.pack(pady=10)

    def add_question_ui(self):
        self.clear_frame()

        self.question_label = tk.Label(self.frame, text="Enter Question:")
        self.question_label.grid(row=0, column=0, pady=5)

        self.question_entry = tk.Entry(self.frame, width=50)
        self.question_entry.grid(row=0, column=1, pady=5)

        self.choice_labels = []
        self.choice_entries = []
        for i in range(3):
            choice_label = tk.Label(self.frame, text=f"Choice {i+1}:")
            choice_label.grid(row=i+1, column=0, pady=5)
            self.choice_labels.append(choice_label)

            choice_entry = tk.Entry(self.frame, width=50)
            choice_entry.grid(row=i+1, column=1, pady=5)
            self.choice_entries.append(choice_entry)

        self.answer_label = tk.Label(self.frame, text="Correct Answer (1, 2, or 3):")
        self.answer_label.grid(row=4, column=0, pady=5)

        self.answer_entry = tk.Entry(self.frame, width=50)
        self.answer_entry.grid(row=4, column=1, pady=5)

        self.submit_button = tk.Button(self.frame, text="Submit", command=self.add_question)
        self.submit_button.grid(row=5, column=0, columnspan=2, pady=10)

        self.back_button = tk.Button(self.frame, text="Back", command=self.setup_ui)
        self.back_button.grid(row=6, column=0, columnspan=2, pady=10)

    def add_question(self):
        question = self.question_entry.get()
        choices = [entry.get() for entry in self.choice_entries]
        answer = self.answer_entry.get()

        if question and all(choices) and answer in ["1", "2", "3"]:
            self.questions.append({"question": question, "choices": choices, "answer": int(answer) - 1})
            self.save_questions()
            messagebox.showinfo("Success", "Question added successfully!")
            self.clear_frame()
            self.setup_ui()
            self.start_quiz_button.config(state=tk.NORMAL)
        else:
            messagebox.showerror("Error", "Please fill out all fields correctly.")

    def start_quiz(self):
        self.clear_frame()
        self.current_question = 0
        self.score = 0
        self.show_question()

    def show_question(self):
        if self.current_question < len(self.questions):
            question = self.questions[self.current_question]
            self.question_label = tk.Label(self.frame, text=question["question"], font=("Arial", 16))
            self.question_label.pack(pady=10)

            self.choice_buttons = []
            for i, choice in enumerate(question["choices"]):
                button = tk.Button(self.frame, text=choice, command=lambda i=i: self.check_answer(i))
                button.pack(pady=5)
                self.choice_buttons.append(button)
        else:
            self.show_score()

    def check_answer(self, selected):
        if selected == self.questions[self.current_question]["answer"]:
            self.score += 1
            messagebox.showinfo("Correct!", "You got it right!")
        else:
            correct_answer = self.questions[self.current_question]["choices"][self.questions[self.current_question]["answer"]]
            messagebox.showerror("Wrong", f"Sorry, that's not correct. The correct answer is: {correct_answer}")
        self.current_question += 1
        self.clear_frame()
        self.show_question()

    def show_score(self):
        self.clear_frame()
        self.score_label = tk.Label(self.frame, text=f"Your Score: {self.score}/{len(self.questions)}", font=("Arial", 20))
        self.score_label.pack(pady=20)

        self.restart_button = tk.Button(self.frame, text="Restart", command=self.setup_ui)
        self.restart_button.pack(pady=10)

    def save_questions(self):
        with open(self.questions_file, 'w') as file:
            json.dump(self.questions, file)

    def load_questions(self):
        if os.path.exists(self.questions_file):
            with open(self.questions_file, 'r') as file:
                return json.load(file)
        return []

    def clear_frame(self):
        for widget in self.frame.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = QuizApp(root)
    root.mainloop()
